### Clone repository and install libraries

In [1]:
import os
os.chdir("/content/")
!rm -rf ASR_Conformer

In [2]:
!git clone https://github.com/Torfinhell/ASR_Conformer

Cloning into 'ASR_Conformer'...
remote: Enumerating objects: 1426, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 1426 (delta 13), reused 25 (delta 11), pack-reused 1382 (from 1)
Receiving objects: 100% (1426/1426), 31.93 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (849/849), done.


In [3]:
#Going inside folder ASR_Conformer
import os
os.chdir("ASR_Conformer")

In [4]:
!pip install uv
!uv sync
!pip install wget

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 63.4 MB/s eta 0:00:00
Using CPython 3.12.0
Creating virtual environment at: .venv
Resolved 194 packages in 4.26s
Prepared 191 packages in 1m 06s
Installed 191 packages in 1.14s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + anyio==4.12.0
 + argparse==1.4.0
 + asttokens==3.0.1
 + attrs==25.4.0
 + audiomentations==0.43.1
 + audioread==3.1.0
 + beautifulsoup4==4.14.3
 + black==25.9.0
 + cachetools==6.2.4
 + certifi==2025.11.12
 + cffi==2.0.0
 + cfgv==3.5.0
 + charset-normalizer==3.4.4
 + click==8.3.1
 + comet-ml==3.53.1
 + comm==0.2.3
 + configobj==5.0.9
 + contourpy==1.3.3
 + cryptography==46.0.3
 + cycler==0.12.1
 + datasets==4.2.0
 + debugpy==1.8.19
 + decorator==5.2.1
 + dill==0.4.0
 + distlib==0.4.0
 + dulwich==0.25.0
 + editdistance==0.8.1
 + einops==0.8.1
 + everett==3.1.0
 + executing==2.2.1
 + filelock==3.20.1
 + fonttools==4.61.1
 + frozenlist

In [5]:
#to create a run that shows augmentations
# !uv run scripts/show_augs.py

### Download checkpoints and dataset

In [6]:
#downloading all checkpoints and example dataset with this command
# !uv run scripts/download_gdrive.py

In [11]:
#Or specify your own links like in the file and call function for downloading
from scripts.download_gdrive import download_dataset, download_models
GDRIVE_URLS = {
    "models": {
    #     "https://drive.google.com/uc?id=1a1gjSXB3mMsNOHcdndhJW6ABZ8HzBEFm": "data/models/chk_bpeablation.pth",  #this is not important to download, just an example
        "https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1": "data/models/chk_train-other-500.pth"   # train-other-500
    },
    "dataset": {# YOUR LINK HERE FOR DATASET
        "https://drive.google.com/uc?id=1ZEqEX6s7lWvCqBRclKyRW4TwJQFcxH4F": "data/datasets/with_gt",
        "https://drive.google.com/uc?id=1rEynDEqEaYSL-2tI7dLTuXAPzLDCl-VY": "data/datasets/without_gt"
    }
}
download_models(GDRIVE_URLS)
download_dataset(GDRIVE_URLS)

Downloading...
From (original): https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1
From (redirected): https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1&confirm=t&uuid=c5339ad6-2b22-457c-b995-bd0fb7df2d78
To: /content/ASR_Conformer/data/models/chk_train-other-500.pth
100%|██████████| 114M/114M [00:00<00:00, 171MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZEqEX6s7lWvCqBRclKyRW4TwJQFcxH4F
To: /content/ASR_Conformer/data/datasets/with_gt.zip
100%|██████████| 747k/747k [00:00<00:00, 152MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rEynDEqEaYSL-2tI7dLTuXAPzLDCl-VY
To: /content/ASR_Conformer/data/datasets/without_gt.zip
100%|██████████| 745k/745k [00:00<00:00, 120MB/s]


### Inference

##### inference on test_other librispeech dataset

In [12]:
#Inferences model on test-other  dataset and produces metrics
# Also train-clean-100 is downloaded to create the bpe vocabulary
#beam size can be reduced for faster inference

from pathlib import Path
model_dir = "data/models"
output_dir="test_other_res"

for model_path in Path(model_dir).glob("*.pth"):
    print(f"{model_path} inferenced in :\n")
    if(str(model_path)=="data/models/chk_bpeablation.pth"):
        !uv run inference.py \
            inferencer.from_pretrained={model_path} text_encoder=CTCEncoder \
            inferencer.save_path={output_dir} text_encoder.beam_size=100 \
            -cn=inference_all_metrics
    else:
        !uv run inference.py \
            inferencer.from_pretrained={model_path} text_encoder=BpeEncoder \
            inferencer.save_path={output_dir} text_encoder.beam_size=100 \
            -cn=inference_all_metrics


data/models/chk_bpeablation.pth inferenced in :

Conformer(
  (conv_subsampling): Conv2dSubsampling(
    (subsampling): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (3): ReLU()
    )
  )
  (linear1): Linear(in_features=19, out_features=144, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (conformer_blocks): Sequential(
    (0): ConformerBlock(
      (ffn1): FeedForwardNet(
        (sequential): Sequential(
          (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=144, out_features=576, bias=True)
          (2): SwishActivation()
          (3): Dropout(p=0.1, inplace=False)
          (4): Linear(in_features=576, out_features=144, bias=True)
          (5): Dropout(p=0.1, inplace=False)
        )
      )
      (mhsa): MultiHeadSelfAttention(
        (layer_norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (pos_

##### inference on dataset dir with and without transcrptions

In [13]:
#this way we inference our models on dataset_dir
model_dir = "data/models"
from pathlib import Path
#choose type with gt or without and specify your dataset_dir, dataset_dir doesnt have to depend on gt_name
gt_name="with_gt"
# gt_name="without_gt" #metrics will return 0.0
dataset_dir = f"data/datasets/{gt_name}/test_data"
for model_path in Path(model_dir).glob("*.pth"):
    print(f"{model_path} inferenced in :\n")
    if(str(model_path)=="data/models/chk_bpeablation.pth"):
        !uv run inference.py dataloader=onebatchtest \
            inferencer.dataset_dir={dataset_dir} \
            inferencer.from_pretrained={model_path} \
            inferencer.save_path={gt_name} \
            text_encoder=CTCEncoder -cn=inference
    else:
        !uv run inference.py dataloader=onebatchtest \
            inferencer.dataset_dir={dataset_dir} \
            inferencer.from_pretrained={model_path} \
            inferencer.save_path={gt_name} \
            text_encoder=BpeEncoder -cn=inference

data/models/chk_bpeablation.pth inferenced in :

Conformer(
  (conv_subsampling): Conv2dSubsampling(
    (subsampling): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (3): ReLU()
    )
  )
  (linear1): Linear(in_features=19, out_features=144, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (conformer_blocks): Sequential(
    (0): ConformerBlock(
      (ffn1): FeedForwardNet(
        (sequential): Sequential(
          (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=144, out_features=576, bias=True)
          (2): SwishActivation()
          (3): Dropout(p=0.1, inplace=False)
          (4): Linear(in_features=576, out_features=144, bias=True)
          (5): Dropout(p=0.1, inplace=False)
        )
      )
      (mhsa): MultiHeadSelfAttention(
        (layer_norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (pos_

### Calculate Metrics

In [14]:
#produces cer and wer metrics for two folders with preds and gt texts
prediction_path=f"data/saved/{gt_name}"
ground_truth_path=f"data/datasets/{gt_name}/test_data/transcriptions"
!uv run scripts/calculate_wer_cer.py prediction_path={prediction_path} ground_truth_path={ground_truth_path}

CER Mean is 0.02988238907058401 and WER Mean is 0.09003463203463202
